# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import packages

In [2]:
import os
import cv2
from skimage.metrics import structural_similarity as ssim

# Calculate SSIM

In [3]:
# Define paths to the original and generated image folders
original_path = '/content/drive/MyDrive/CS245 Project/x-ray-edges-json/Test/target'
generated_path_nofinetune = '/content/drive/MyDrive/CS245 Project/ControlNet_UnFinetune_TestOutput'
generated_path_finetune_5 = '/content/drive/MyDrive/CS245 Project/ControlNet_Finetune_TestOutput_5'
generated_path_finetune_25 = '/content/drive/MyDrive/CS245 Project/ControlNet_Finetune_TestOutput_25'
generated_path_finetune_50 = '/content/drive/MyDrive/CS245 Project/ControlNet_Finetune_TestOutput_50'

generated_paths = [
    generated_path_nofinetune,
    generated_path_finetune_5,
    generated_path_finetune_25,
    generated_path_finetune_50
]

In [4]:
# Calculate ssim image vs image
def calculate_ssim(img1, img2):
    # Convert images to grayscale if they are in color
    if img1.shape[-1] == 3:
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    if img2.shape[-1] == 3:
        img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Ensure the images have the same dimensions
    min_height = min(img1.shape[0], img2.shape[0])
    min_width = min(img1.shape[1], img2.shape[1])
    img1 = img1[:min_height, :min_width]
    img2 = img2[:min_height, :min_width]

    # Calculate SSIM
    score, _ = ssim(img1, img2, full=True)

    return score


# Compare images from two folders
def compare_images(folder1, folder2):
    ssim_scores = []

    images1 = [cv2.imread(os.path.join(folder1, filename)) for filename in sorted(os.listdir(folder1))]
    images2 = [cv2.imread(os.path.join(folder2, filename)) for filename in sorted(os.listdir(folder2))]

    for img1, img2 in zip(images1, images2):
        ssim_score = calculate_ssim(img1, img2)
        ssim_scores.append(ssim_score)

    # Calculate the average SSIM score
    avg_ssim = sum(ssim_scores) / len(ssim_scores)

    return avg_ssim

In [5]:
# Compare images and calculate average SSIM score for each path
for path in generated_paths:
    avg_ssim = compare_images(original_path, path)
    print(f"Average SSIM (Original vs {path.split('/')[-1]}): {avg_ssim:.2f}")

Average SSIM (Original vs ControlNet_UnFinetune_TestOutput): 0.38
Average SSIM (Original vs ControlNet_Finetune_TestOutput_5): 0.59
Average SSIM (Original vs ControlNet_Finetune_TestOutput_25): 0.54
Average SSIM (Original vs ControlNet_Finetune_TestOutput_50): 0.58
